# Asteroid Tracking Workbook

This workbook will guide you through measuring the precise RA and Dec coordinates (astrometry) of the asteroid 2013 GG69 on the night of June 19th, 2022. After making your measurements, you will input them into an orbit solver to visualize the orbit and find how frequently and closely the asteroid approaches Earth.

### Imports
We need a relatively large number of packages for this workbook. These packages include functions that will do most of the really hard work for us, including measuring star positions and performing astrometric solving. The next cell includes all imports.

In [1]:
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.wcs import WCS
from astropy.time import Time
from datetime import datetime

from astroquery.astrometry_net import AstrometryNet

from photutils.aperture import CircularAperture
from photutils.centroids import centroid_2dg
from photutils.centroids import centroid_sources

import matplotlib.pyplot as plt
from matplotlib import colors

from sow_tools import mask_bad_pix

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


The below cell is the base path where all the files for this project are located. Do not edit.

In [2]:
store_path = 'raw/'

We need to get the names of all the files we will be using. Edit the list definitions below with correct file names and locations.
e.g. bias_files = ['file1.fits', 'file2.fits', etc...]

In [3]:
bias_files = ['d1022.fits', 'd1023.fits', 'd1024.fits', 'd1025.fits', 'd1026.fits', 'd1027.fits', 'd1028.fits', 'd1029.fits', 'd1030.fits']
flat_files = ['d1037.fits', 'd1038.fits', 'd1039.fits'] 
sci_files = ['d1065.fits', 'd1066.fits', 'd1067.fits', 'd1068.fits', 'd1069.fits', 'd1070.fits'] # science images that include asteroid

### Creating main calibrations
We want to average our individual calibration images together to make a main image that is more accurate than any of the individual images. To do this, we will read in the image data from each of our individual calibrations then take the median of every pixel value. It is better to take the median instead of a true average because the median is not changed by one large outlier, while an average is.

### Edit the code below to create a main bias image and save it in the main_bias variable
Reference the data reduction workbook or ask an instructor for help if you aren't sure what to do.

In [4]:
bias_data = np.empty((9,1024,1056))
for i, file in enumerate(bias_files):
    hdu = fits.open(store_path+file)
    bias_data[i] = hdu[0].data
    hdu.close()

main_bias = np.nanmedian(bias_data,axis=0)

In [5]:
%matplotlib widget

med_bval = np.nanmedian(main_bias)

plt.figure(figsize=(10,10))
plt.imshow(main_bias, vmin=med_bval*0.98, vmax=med_bval*1.02, origin='lower')
plt.colorbar()
plt.title('Main bias image', fontsize=20)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now do the same but for the flat images. Save the image in the main_flat variable.
Remember that each individual flat image needs the main_bias subtracted from it before they are combined together.

In [6]:
flat_data = np.empty((3,1024,1056))
for i, file in enumerate(flat_files):
    hdu = fits.open(store_path+file)
    bias_sub = (hdu[0].data - main_bias)
    
    flat_data[i] = bias_sub / np.nanmedian(bias_sub)
    hdu.close()

main_flat = np.nanmedian(flat_data, axis=0)

In [7]:
%matplotlib widget

med_fval = np.nanmedian(main_flat)

plt.figure(figsize=(10,10))
plt.imshow(main_flat, vmin=med_fval*0.9, vmax=med_fval*1.1, origin='lower')
plt.colorbar()
plt.title('Main flat image', fontsize=20)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Correcting our science images with the main calibrations
Here we will use our main bias and main flat images to correct all of the science images containing the asteroid. The science images are loaded for you, but you need to input the math for correcting it into the loop. 
In addition, we will use a function **mask_bad_pix** to remove bax pixels and cosmic rays from our science images. This function is already written, you simply need to give it the corrected image as the **sci_im** variable.

At the end of this cell, we save important information from the .fits header including the time that the observations occurred and where the telescope was roughly pointing in RA and Dec.

In [ ]:
num_sci = len(sci_files)
sci_data = np.empty((num_sci,1024,1024))
sci_coords = np.empty((num_sci,2))
time_data = []
for i, file in enumerate(sci_files):
    # loop through all science files
    hdu = fits.open(store_path+file)
    
    sci_im = hdu[0].data ### FIXME ###
    
    # mask_bad_pix will search for significant outliers in our data and remove them, we also remove an unneeded overscan region of the images
    sci_data[i] = mask_bad_pix(sci_im)[0:1024,0:1024]
    
    # write the corrected science files into new .fits files that will be saved
    tmp = fits.PrimaryHDU(sci_data[i], header=hdu[0].header)
    tmpl = fits.HDUList(tmp)
    tmpl.writeto(store_path+file.strip('.fits')+'_calib.fits', overwrite=True)
    tmpl.close()
    
    # we also want to save some values from the .fits header that we need for astrometric measurements
    # grab the approximate RA and Dec pointing of the telescope
    sci_coords[i] = hdu[0].header['CRVAL2S'], hdu[0].header['CRVAL1S'] 
    # get the beginning and end time of each exposure
    time_data.append([Time(hdu[0].header['DATE-BEG'], format='isot', scale='utc'), Time(hdu[0].header['DATE-END'], format='isot', scale='utc')]) 
    
    # close and go to the next image
    hdu.close()

### Guessing star locations for an astrometric image solution
In the next few cells, we will work on providing astrometry.net the information it needs to give us an astrometric solution for each of our images. The most fundamental information that astrometry.net is the pixel positions of all the stars in the image. We could use a package to automatically identify all the stars, however, this can be somewhat difficult because if we accidentally identify some random noise as a star then it will be impossible be astrometry.net to find a solution since we've given it one or more stars that don't actually exist.

To be safer, we can manually identify the approximate pixel locations of sources we know are stars, then refine their positions with another package before giving them to astrometry.net.

### pyds9 TBD

In [ ]:
prelim_sources = []#FIXME#FIXME#FIXME

prelim_sources = np.array(prelim_sources)

#### Telling astrometry.net our CCD scale
astrometry.net will find a solution much faster if we tell it the dimensions of our images and the rough pixel scale (what fraction of an arcsecond each pixel covers)

Find this information in the manual for the Nickel Direct Imaging Camera - https://mthamilton.ucolick.org/techdocs/instruments/nickel_direct/intro/

**Important -** all our images are taken with 2x pixel binning, which means the size of the detector is reduced by half and the pixel scale is doubled.

In [ ]:
image_width = #FIXME
image_height = #FIXME

plate_scale = #FIXME

Using astrometry.net requires an API key associated to a registered account. This is provided below.

In [ ]:
api = 'svigftgopyhezcqs'
ast = AstrometryNet()
ast.api_key = api

### Refining sources and getting astrometric solutions!
Now we are ready to loop through our images and get an astrometric solution for each one. The next cell contains the loop.

At the beginning of the loop, we first use the centroid_sources function to precisely refine the pixel locations of stars we identified previously. For every image, we will go ahead and plot a red circle around the location of identified stars, that we can see if any stars are detected that shouldn't be.

Next, we will get the astrometric solution with the ast.solve_from_source_list() function. Here we pass it the star positions and the CCD parameters that we also found earlier. We also give it the rough RA and Dec pointing of the telescope so that astrometry.net doesn't have to search the entire sky for a matching pattern of stars. In general, astrometry.net can find solutions with very little information, but it goes much faster the more information we provide it.

ast.solve_from_source_list() returns the astrometry in a World Coordinate System (WCS) 'header' format, which we will put into a list for every image. These headers don't directly contain the astrometry for our asteroid, but contain all the information we need later. 

In [ ]:
wcs_list = []
    
for i, data in enumerate(sci_data):
    
    x, y = centroid_sources(data, prelim_sources[:,0], prelim_sources[:,1], box_size=21,
                        centroid_func=centroid_2dg)

    positions = np.transpose((x, y))
    apertures = CircularAperture(positions, r=4.0)
    plt.figure(figsize=(10,10))
    plt.imshow(data, origin='lower', vmin=0, vmax=30,
               interpolation='nearest')
    apertures.plot(color='red', lw=1.5, alpha=1.0)
    plt.colorbar()
    plt.show()
    
    wcs_header = ast.solve_from_source_list(x, y,
                                            image_width, image_height,
                                            solve_timeout=120, scale_units='arcsecperpix', scale_type='ev', scale_est=scale_est, scale_err=0.01,
                                            center_dec=sci_coords[i,0], center_ra=sci_coords[i,1], radius=1.0, parity=2)
    wcs_list.append(wcs_header)
    

The next cell is to check that all our astrometric solutions were successful. **If an error is returned, at least one solution was not successful** - get assistance from the instructor to identify what is going wrong before continuing.

In [ ]:
assert any(elem is None for elem in wcs_list) == False

### Measuring the asteroid positions
We have an astrometric solution for our images, but we still need to measure the actual pixel positions of our asteroid to convert to RA and Dec with the solutions.

Below we will once again loop through our images and use the DAOStarFinder function to find the asteroid position. 

To make this easier, **set a range of x and y values in the image that will only contain the asteroid and no other sources**. The upper range and lower ranges correspond to the top and bottom pixel values that will bound the box containing our asteroid.

As before, we will plot a red circle on the identified source in each image. **Check that the circle is on the asteroid for every plot**.

In [ ]:
x_lower_range = #FIXME
x_upper_range = #FIXME
y_lower_range = #FIXME
y_upper_range = #FIXME

src_list = []

for i, data in enumerate(sci_data):
    strip = data[y_lower_range:y_upper_range, x_lower_range:x_upper_range]
    mean, median, std = sigma_clipped_stats(strip, sigma=3.0)

    daofind = DAOStarFinder(fwhm=5.0, threshold=5.*std)  
    sources = daofind(strip - median) 
    src_list.append([sources['xcentroid'], sources['ycentroid']])
    
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    apertures = CircularAperture(positions, r=4.0)
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(strip, origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='red', lw=1.5, alpha=1.0)
    plt.show()

### Converting position to asteroid astrometry